In [50]:
import json, os

In [2]:
with open("images/det_ingrs.json", "r") as f:
    data = json.load(f)

In [3]:
len(data)

1029720

In [43]:
data[303]

{'valid': [True, True, True, True, True, True, True, True, True, True],
 'id': '00141df42c',
 'ingredients': [{'text': 'boneless skinless chicken breasts'},
  {'text': 'garlic powder'},
  {'text': 'eggs'},
  {'text': 'all - purpose flour'},
  {'text': 'sugar substitute'},
  {'text': 'pineapple chunks'},
  {'text': 'white vinegar'},
  {'text': 'ketchup'},
  {'text': 'low sodium soy sauce'},
  {'text': 'salt'}]}

In [5]:
with open("images/layer1.json", "r") as f:
    dataLayer1 = json.load(f)
    
len(dataLayer1)

1029720

In [42]:
dataLayer1[303]

{'ingredients': [{'text': '4 boneless skinless chicken breasts'},
  {'text': '12 teaspoon garlic powder'},
  {'text': '2 eggs, beaten'},
  {'text': '12 cup all-purpose flour'},
  {'text': '12 cup sugar substitute'},
  {'text': '1 (8 ounce) can pineapple chunks, drained, with 1/4 cup juice reserved'},
  {'text': '34 cup white vinegar'},
  {'text': '34 cup ketchup'},
  {'text': '1 teaspoon low sodium soy sauce'},
  {'text': '34 teaspoon salt'}],
 'url': 'http://www.food.com/recipe/pineapple-chicken-180582',
 'partition': 'val',
 'title': 'Pineapple Chicken',
 'id': '00141df42c',
 'instructions': [{'text': 'Heat oven to 350.'},
  {'text': 'Spray 9x13 pan with cooking spray.'},
  {'text': 'Season chicken with garlic powder.'},
  {'text': 'Dip chicken in egg, then flour.'},
  {'text': 'Put chicken in pan.'},
  {'text': 'In saucepan, mix together sugar, pineapple juice, vinegar, ketchup, soy, and salt.'},
  {'text': 'Bring to boil.'},
  {'text': 'Pour mixture over chicken and arrange pineapp

In [98]:
train_IDs, test_IDs, valid_IDs = [], [], []
layer1Data = []
for k in dataLayer1:
    if k["partition"] == "train":
        train_IDs.append(k["id"])
    if k["partition"] == "test":
        test_IDs.append(k["id"])
    if k["partition"] == "val":
        valid_IDs.append(k["id"])
        
    layer1Data.append(k)
        
    if len(layer1Data) == 1000:
        break

In [99]:
len(valid_IDs), len(train_IDs), len(test_IDs), len(layer1Data)

(148, 698, 154, 1000)

In [7]:
with open("images/layer2.json", "r") as f:
    dataLayer2 = json.load(f)
    
len(dataLayer2)

402760

In [26]:
dataLayer2[2340]

{'id': '01749ea781',
 'images': [{'id': 'b31c3c48ca.jpg',
   'url': 'http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/16/41/27/picbboqWt.jpg'}]}

In [100]:
layer2Data = []
for k in dataLayer2:
    if k["id"] in train_IDs + test_IDs + valid_IDs:
        layer2Data.append(k)

In [104]:
import urllib.request, requests
def downloadImage(imgURL, path):
    try:
        response = requests.get(imgURL)
        with open(path, "wb") as f:
            f.write(response.content)
    except Exception as e:
        pass

In [107]:
os.system(f"mkdir images/val/")
os.system(f"mkdir images/train/")
os.system(f"mkdir images/test/")

for k in dataLayer2:
    if k["id"] in valid_IDs:
        # print(k)
        foodId = k["id"]
        allImages = k["images"]
        os.mkdir(f"images/val/{foodId}")
        for im in allImages:
            downloadImage(im["url"], f"images/val/{foodId}/{im['id']}")
            
    if k["id"] in train_IDs:
        # print(k)
        foodId = k["id"]
        allImages = k["images"]
        os.mkdir(f"images/train/{foodId}")
        for im in allImages:
            downloadImage(im["url"], f"images/train/{foodId}/{im['id']}")
            
    if k["id"] in test_IDs:
        # print(k)
        foodId = k["id"]
        allImages = k["images"]
        os.mkdir(f"images/test/{foodId}")
        for im in allImages:
            downloadImage(im["url"], f"images/test/{foodId}/{im['id']}")
            
walk = list(os.walk("../images/train/"))
for path, _, _ in walk[::-1]:
    if len(os.listdir(path)) == 0:
        os.remove(path)
        
walk = list(os.walk("../images/test/"))
for path, _, _ in walk[::-1]:
    if len(os.listdir(path)) == 0:
        os.remove(path)
        
walk = list(os.walk("../images/val/"))
for path, _, _ in walk[::-1]:
    if len(os.listdir(path)) == 0:
        os.remove(path)

In [36]:
# 36ad774009, no images

In [108]:
dataSmall = []
for d in data:
    if d["id"] in train_IDs + test_IDs + valid_IDs:
        dataSmall.append(d)
        
len(dataSmall)

1000

In [109]:
len(layer1Data), len(layer2Data), len(dataSmall)

(1000, 401, 1000)

In [110]:
with open("images/det_ingrs_small.json", "w+") as f:
    json.dump(dataSmall, f)
    
with open("images/layer1_small.json", "w+") as f:
    json.dump(layer1Data, f)
    
with open("images/layer2_small.json", "w+") as f:
    json.dump(layer2Data, f)

In [111]:
for i in range(len(dataSmall)):
    if dataSmall[i]["id"] != layer1Data[i]["id"]:
        print("error")

In [130]:
import pickle

# open pickle file in read binary mode
with open('garbage/recipe1m_vocab_toks.pkl', 'rb') as f:
    # load the data from the file using pickle.load()
    dataTemp = pickle.load(f)
    
type(dataTemp), len(dataTemp)

AttributeError: Can't get attribute 'Vocabulary' on <module '__main__'>

In [128]:
dataTemp[0]["title"]

['crunchy', 'onion', 'potato', 'bake']

In [ ]:
python train.py --model_name im2ingr --batch_size 150 --finetune_after 0 --ingrs_only \
--es_metric iou_sample --loss_weight 0 1000.0 1.0 1.0 \
--learning_rate 1e-4 --scale_learning_rate_cnn 1.0 \
--save_dir ../checkpoints --recipe1m_dir path_to_dataset --load_jpeg